## Imports

In [ ]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [ ]:
CLOUD = True

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "umutakta"
    os.environ['KAGGLE_KEY']      = "a471e5474add8d57cadac3cc5ff2b1d6"  # See https://www.kaggle.com/docs/api
    #!pip install --upgrade kaggle
    #!kaggle competitions download -c titanic
    DATA_PATH = "/work/Strive_School/Feature_Engineering_(Module_4)/Day_4_ Categorical_Encoding/titanic/"

else:
    DATA_PATH = "../../datasets/titanic/"

DATA_PATH

'/work/Strive_School/Feature_Engineering_(Module_4)/Day_4_ Categorical_Encoding/titanic/'

## Load data

In [ ]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [ ]:
#df.isnull().sum(),
df_test.isnull().sum()

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        1
Cabin       0
Embarked    0
Title       1
dtype: int64


Extract the title (Mr, Mrs, ... ) from the "Name" column.


In [ ]:

get_Title_from_Name = lambda x: x.split(",")[1].split(".")[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name )
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [ ]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [ ]:
df["Title"] = df['Title'].map(title_dictionary)
df_test["Title"] = df_test['Title'].map(title_dictionary)

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [ ]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [ ]:
df_test['Cabin']

df.Cabin.fillna(0, inplace=True) #We named Null cells as 'Empty'
df_test.Cabin.fillna(0, inplace=True) #We named Null cells as 'Empty'

df.loc[df["Cabin"] != 0, "Cabin"] = 1
df_test.loc[df_test["Cabin"] != 0, "Cabin" ] =1


df_test["Fare"]=df_test["Fare"].fillna(value=df_test["Fare"].mean())

df_test.isnull().sum()


Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
Title       1
dtype: int64

### Target Encoding

In [ ]:
import category_encoders as ce
target_encoder = ce.TargetEncoder(smoothing=0)

train_encoded=target_encoder.fit_transform(df["Embarked"],df["Survived"])
df["Embarked"]=train_encoded
df["Embarked"]

/root/venv/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


PassengerId
1      0.336957
2      0.553571
3      0.336957
4      0.336957
5      0.336957
         ...   
887    0.336957
888    0.336957
889    0.336957
890    0.553571
891    0.389610
Name: Embarked, Length: 891, dtype: float64

In [ ]:
df.Age.fillna(df.groupby(['Pclass', 'Sex']).Age.transform('mean'), inplace=True)
df_test.Age.fillna(df_test.groupby(['Pclass', 'Sex']).Age.transform('mean'), inplace=True)


# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [ ]:
x = df.drop(columns=["Survived", 'Name', 'Ticket']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket']) # # X_TEST DATA (NEW DATA)

In [ ]:
cat_vars  = ['Sex', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age', 'Embarked'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age', 'Embarked']

Categorical features:
 ['Sex', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [ ]:

#num_4_treeModels = pipeline.Pipeline(steps=[('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)), 
#                                            ])

#cat_4_treeModels = pipeline.Pipeline(steps=[
#    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
#    ('ordinal', preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan
#))
#])

cat_4_multModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('one hot', preprocessing.OneHotEncoder(handle_unknown="ignore"
))
])


num_4_multmodels=pipeline.Pipeline(steps=[
    ("imputer",KNNImputer(n_neighbors=5)),
    ("quantile transformer",preprocessing.QuantileTransformer(n_quantiles=100, random_state=0))
    ])

tree_prepro = compose.ColumnTransformer(transformers=[
    #('num', num_4_treeModels, num_vars),
    #('cat', cat_4_treeModels, cat_vars),
    ("cat mult",cat_4_multModels,cat_vars),
    ("num mult",num_4_multmodels,num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro



ColumnTransformer(transformers=[('cat mult',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('one hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Title']),
                                ('num mult',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('quantile transformer',
                                                  QuantileTransformer(n_quantiles=100,
                                                                      random_state=0))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age',
                                  'Embarked'])])

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [ ]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [ ]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
mult_classifiers={
    "SVM": svm.SVC(),
    "Logistic": LogisticRegression(),
    "KNN":KNeighborsClassifier(n_neighbors=3),
    "MLP":MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)}

mult_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in mult_classifiers.items()}

mult_classifiers["SVM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Title']),
                                                 ('num mult',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('quantile '
                                                                   'transformer',
                                                                   QuantileTransformer(n_quantiles=100,
                                                                                       random_state=0))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age',
                                                   'Embarked'])])),
                ('svc', SVC())])

In [ ]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in mult_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

,Model,Accuracy,Bal Acc.,Time
1,MLP,84.916201,82.325428,0.236838
2,Logistic,83.240223,81.501976,0.042139
3,SVM,82.681564,79.696970,0.039446
4,KNN,80.446927,78.418972,0.045287


In [ ]:

from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})





for model_name, model in mult_classifiers.items():
    start_time = time.time()


        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x,y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# YOUR CODE HERE



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (

,Model,Accuracy,Bal Acc.,Time
1,MLP,81.593715,79.220592,1.865240
2,SVM,81.369248,78.211581,0.447945
3,Logistic,81.144781,79.242163,0.666128
4,KNN,79.797980,78.204391,0.294117


In [ ]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": GradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier() }


# YOUR CODE HERE


tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Title']),
                                                 ('num mult',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('quantile '
                                                                   'transformer',
                                                                   QuantileTransformer(n_quantiles=100,
                                                                                       random_state=0))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age',
                                                   'Embarked'])])),
                ('lgbmclassifier', LGBMClassifier())])

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [ ]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


1:	learn: 0.6818340	total: 49.1ms	remaining: 24.5s
2:	learn: 0.6765413	total: 50.2ms	remaining: 16.7s
3:	learn: 0.6707946	total: 51.2ms	remaining: 12.8s
4:	learn: 0.6657201	total: 52.1ms	remaining: 10.4s
5:	learn: 0.6625543	total: 52.6ms	remaining: 8.72s
6:	learn: 0.6576241	total: 53.6ms	remaining: 7.61s
7:	learn: 0.6531408	total: 54.4ms	remaining: 6.74s
8:	learn: 0.6496930	total: 55.3ms	remaining: 6.09s
9:	learn: 0.6467767	total: 55.8ms	remaining: 5.53s
10:	learn: 0.6418560	total: 56.8ms	remaining: 5.11s
11:	learn: 0.6371246	total: 57.9ms	remaining: 4.76s
12:	learn: 0.6319074	total: 59ms	remaining: 4.48s
13:	learn: 0.6275882	total: 60ms	remaining: 4.22s
14:	learn: 0.6226872	total: 61ms	remaining: 4.01s
15:	learn: 0.6182087	total: 62.1ms	remaining: 3.82s
16:	learn: 0.6142786	total: 62.8ms	remaining: 3.63s
17:	learn: 0.6097641	total: 63.8ms	remaining: 3.48s
18:	learn: 0.6064363	total: 64.4ms	remaining: 3.33s
19:	learn: 0.6024827	total: 65.4ms	remaining: 3.21s
20:	learn: 0.6001947	total:

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,86.592179,85.309618,1.306856
2,Skl GBM,85.474860,83.860343,0.133919
3,Skl HistGBM,85.474860,83.860343,0.117510
4,LightGBM,83.798883,83.036891,0.091135
5,AdaBoost,83.240223,82.852437,0.098936
6,XGBoost,82.681564,81.857708,0.169271
7,Random Forest,82.122905,79.782609,0.176670
8,Extra Trees,81.005587,78.873518,0.148689
9,Decision Tree,79.329609,78.050066,0.018147


# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [ ]:

from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})





for model_name, model in tree_classifiers.items():
    start_time = time.time()


        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x,y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# YOUR CODE HERE



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')
print("done")

2:	learn: 0.6747618	total: 3.33ms	remaining: 1.11s
3:	learn: 0.6687106	total: 4.47ms	remaining: 1.11s
4:	learn: 0.6635666	total: 5.17ms	remaining: 1.03s
5:	learn: 0.6578367	total: 6.26ms	remaining: 1.04s
6:	learn: 0.6522758	total: 7.39ms	remaining: 1.05s
7:	learn: 0.6468097	total: 8.55ms	remaining: 1.06s
8:	learn: 0.6415641	total: 9.62ms	remaining: 1.06s
9:	learn: 0.6356896	total: 10.7ms	remaining: 1.06s
10:	learn: 0.6301142	total: 11.8ms	remaining: 1.06s
11:	learn: 0.6248813	total: 12.9ms	remaining: 1.06s
12:	learn: 0.6202576	total: 13.8ms	remaining: 1.05s
13:	learn: 0.6151900	total: 14.9ms	remaining: 1.05s
14:	learn: 0.6096069	total: 16.1ms	remaining: 1.06s
15:	learn: 0.6056472	total: 16.8ms	remaining: 1.03s
16:	learn: 0.6010461	total: 17.9ms	remaining: 1.04s
17:	learn: 0.5964921	total: 19ms	remaining: 1.04s
18:	learn: 0.5920382	total: 20.3ms	remaining: 1.05s
19:	learn: 0.5877664	total: 21.4ms	remaining: 1.05s
20:	learn: 0.5832608	total: 22.5ms	remaining: 1.05s
21:	learn: 0.5792769	t

# Exercise 7.1
Train with all data the best model

In [ ]:
best_model = tree_classifiers["Skl GBM"]

# Fit best model with all data
best_model.fit(x,y)



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Title']),
                                                 ('num mult',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('quantile '
                                                                   'transformer',
                                                                   QuantileTransformer(n_quantiles=100,
                                                                                       random_state=0))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age',
                                                   'Embarked'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [ ]:
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.2500,0,0.336957,Mr
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.2833,1,0.553571,Mrs
3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.9250,0,0.336957,Miss
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.1000,1,0.336957,Mrs
5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.0500,0,0.336957,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.00,0,0,211536,13.0000,0,0.336957,Officer
888,1,1,"Graham, Miss. Margaret Edith",female,19.00,0,0,112053,30.0000,1,0.336957,Miss
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.75,1,2,W./C. 6607,23.4500,0,0.336957,Miss


In [ ]:
test_pred = best_model.predict(x_test)


ValueError: Found unknown categories ['missing'] in column 1 during transform

In [ ]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

NameError: name 'test_pred' is not defined

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [ ]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0


In [ ]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

100% 2.77k/2.77k [00:08<00:00, 338B/s]
Successfully submitted to Titanic - Machine Learning from Disaster

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=796ba814-f632-4502-b0d7-7e2f8f9e546d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>